# Sample Hangul RNN

In [1]:
# -*- coding: utf-8 -*-
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import string
import argparse
import time
import os
from six.moves import cPickle
from TextLoader import *
from Hangulpy import *
print ("Packages Imported")

Packages Imported


# Load dataset using TextLoader

In [2]:
corpus_name = "invisible_dragon" # "nine_dreams"
# corpus_name = "nine_dreams"

data_dir    = "data/" + corpus_name
batch_size  = 10
seq_length  = 100
data_loader = TextLoader(data_dir, batch_size, seq_length)
# This makes "vocab.pkl" and "data.npy" in "data/nine_dreams"   
#  from "data/nine_dreams/input.txt" 
vocab_size = data_loader.vocab_size
vocab = data_loader.vocab
chars = data_loader.chars
print ( "type of 'data_loader' is %s, length is %d" 
       % (type(data_loader.vocab), len(data_loader.vocab)) )
print ( "\n" )
print ("data_loader.vocab looks like \n%s " %
       (data_loader.vocab))
print ( "\n" )
print ( "type of 'data_loader.chars' is %s, length is %d" 
       % (type(data_loader.chars), len(data_loader.chars)) )
print ( "\n" )
print ("data_loader.chars looks like \n%s " % (data_loader.chars,))

loading preprocessed files
type of 'data_loader' is <type 'dict'>, length is 76


data_loader.vocab looks like 
{u'_': 69, u'6': 59, u':': 57, u'\n': 19, u'4': 67, u'5': 63, u'>': 75, u'!': 52, u' ': 1, u'"': 28, u'\u1d25': 0, u"'": 49, u')': 46, u'(': 45, u'-': 65, u',': 27, u'.': 24, u'\u3131': 7, u'0': 73, u'\u3133': 60, u'\u3132': 29, u'\u3135': 50, u'\u3134': 4, u'\u3137': 13, u'\u3136': 44, u'\u3139': 5, u'\u3138': 32, u'\u313b': 55, u'\u313a': 48, u'\u313c': 54, u'?': 41, u'3': 66, u'\u3141': 12, u'\u3140': 51, u'\u3143': 47, u'\u3142': 17, u'\u3145': 10, u'\u3144': 43, u'\u3147': 2, u'\u3146': 22, u'\u3149': 40, u'\u3148': 15, u'\u314b': 42, u'\u314a': 23, u'\u314d': 31, u'\u314c': 30, u'\u314f': 3, u'\u314e': 14, u'\u3151': 34, u'\u3150': 21, u'\u3153': 11, u'\u3152': 74, u'\u3155': 18, u'\u3154': 20, u'\u3157': 9, u'\u3156': 39, u'\u3159': 53, u'\u3158': 26, u'\u315b': 38, u'\u315a': 33, u'\u315d': 36, u'\u315c': 16, u'\u315f': 35, u'\u315e': 61, u'\u3161': 8, u'\u3160': 37, 

# Define Network

In [3]:
rnn_size   = 512
num_layers = 3
grad_clip  = 5.

_batch_size = 1
_seq_length = 1

vocab_size = data_loader.vocab_size

with tf.device("/cpu:0"):
    # Select RNN Cell
    def unit_cell():
        return tf.contrib.rnn.BasicLSTMCell(rnn_size,state_is_tuple=True,reuse=tf.get_variable_scope().reuse)
    cell = tf.contrib.rnn.MultiRNNCell([unit_cell() for _ in range(num_layers)])

    # Set paths to the graph 
    input_data = tf.placeholder(tf.int32, [_batch_size, _seq_length])
    targets    = tf.placeholder(tf.int32, [_batch_size, _seq_length])
    initial_state = cell.zero_state(_batch_size, tf.float32)

    # Set Network
    with tf.variable_scope('rnnlm'):
        softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
        softmax_b = tf.get_variable("softmax_b", [vocab_size])
        with tf.device("/cpu:0"):
            embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
            inputs = tf.split(tf.nn.embedding_lookup(embedding, input_data), _seq_length, 1)
            inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
            
    # Loop function for seq2seq
    def loop(prev, _):
        prev = tf.nn.xw_plus_b(prev, softmax_w, softmax_b)
        prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
        return tf.nn.embedding_lookup(embedding, prev_symbol)
    # Output of RNN 
    outputs, last_state = tf.contrib.rnn.static_rnn(cell,inputs, initial_state
                                , scope='rnnlm')
    output = tf.reshape(tf.concat(outputs,1), [-1, rnn_size])
    logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
    # Next word probability 
    probs = tf.nn.softmax(logits)
    final_state = last_state

print ("Network Ready")

Network Ready


In [4]:
# Sample ! 
def sample( sess, chars, vocab, __probs, num=200, prime=u'ㅇㅗᴥㄴㅡㄹᴥ '):
    state = sess.run(cell.zero_state(1, tf.float32))
    _probs = __probs
    prime = list(prime)
    for char in prime[:-1]:
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [state] = sess.run([last_state], feed)

    def weighted_pick(weights):
        weights = weights / np.sum(weights) 
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime
    char = prime[-1]
    for n in range(num):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]
        feed = {input_data: x, initial_state:state}
        [_probsval, state] = sess.run([_probs, final_state], feed)
        p = _probsval[0]
        sample = int(np.random.choice(len(p), p=p))
        #sample = weighted_pick(p)
        pred = chars[sample]
        ret += pred
        char = pred
    return ret
print ("sampling function done.")

sampling function done.


# Sample

In [6]:
save_dir = 'data/' + corpus_name
prime = decompose_text(u" ")

print ("Prime Text : %s => %s" % (automata(prime), "".join(prime)))
n = 4000

sess = tf.Session()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver(tf.all_variables())
ckpt = tf.train.get_checkpoint_state(save_dir)

# load_name = u'data/nine_dreams/model.ckpt-0'
load_name = os.path.join(save_dir,'model.ckpt-99000')

print (load_name)

if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, load_name)
    sampled_text = sample(sess, chars, vocab, probs, n, prime)
    #print ("")
#     print (u"SAMPLED TEXT = %s" % sampled_text)
#     print ("")
    print ("-- RESULT --")
    print (automata("".join(sampled_text)))

Prime Text :   =>  
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please use tf.global_variables instead.
data/nine_dreams/model.ckpt-99000
INFO:tensorflow:Restoring parameters from data/nine_dreams/model.ckpt-99000
-- RESULT --
 그러하도다."
  하였다. 
  태사가 먼저 승상을 모시고 꿇어앉아 사은한 읏을 일어컴기고 구녀를 지어 소뉘가 일어나 나건대 가는 기숙이 묽고
  ㅘㅏ이에게 작은의 몸이 문륵기지 못함이면, 이 상군이 사람을 비어 조그마한 배 
형의 희랑찬이라, 다시 한백 밖에 
도려보지 못하였는데, 이쩨 양소유가 지어 자주 구공문을 듣더니 이를 것 
넌작진자가 봄기 간경차항여 가볍겠고서는, 부친의 황자우로 돌아와 
몸다뱅 앞에 
가히 공주를 헤리려 하소."
 만뜻 사랑으로 밤깝스시면 내가 슨편하시게 돌아오로 
아되에 월업과 깨받이 연끼라 하시니, 상이 여정이 저저를 바라보니, 공주가 
한가의 배하를 진적없게 하지만 그 잔치에게 말하다."
  유부인의 별당으로 돌아오시고 공며(소기와 옥다하였음이데다. 싶삽상이 부슬되었나이다."
  하기에, 춘운이 군사를 듣고 대사로 부처늠꾸로 이번과도 국제호 내일 꺼리지 
않소. 무슨 전이 많을 임금도 얻으면 기대를 
진중하라 행상 제자 동남을 탕부, 못항즙 따라짔다. 
  이에 승상이 정색의 해설하지는 일을 햍치고 죽은 전에 사귀와 
맞아들이게 좋아히 그 산 몸을 굽어들었소이다. 다시 날을 떠로함이 없는 눈갈가 아지 
못하였는데 그 때에 이느로부터늬 부니를 위하여 경추라고 하직인지라. 노조가 
팔선녀에게 맞ㅈ주시고 다휘오지 않사옵거늘 곧 이미 항면 밖 사실이니, 화역히 
간박하고 귀공자 장원은 직제의 의섬으로 더 갑후 펴겨 
주쇠.기를 삼에 

It takes long time to train!